In [ ]:
# This is so you can interact with plots within the notebook
# I.e. zoom, pan, etc.
%matplotlib notebook

In [ ]:
import sys
sys.path.append('/home/ggeva/spt3g/spt3g_software/build')
import os
from spt3g import core, std_processing, mapmaker, dfmux, calibration, todfilter, coordinateutils, timestreamflagging, util
from spt3g.coordinateutils.coordsysmodules import FillCoordTransRotations
import numpy as np
import pylab as plt
from astropy import units
from astropy.coordinates import SkyCoord

In [ ]:
# Flatsky maps that I'm using are 720x720 pixels
X_MAX = 719
Y_MAX = 719

SAVE_FIGS = True
SOURCE = 'RCW38-pixelraster' # Only use pixelraster sources
POLARIZED = False

In [ ]:
# Haven't set up appropriate colorbars for all sources yet.
# Colorbar limits determined by just looking through several maps to find
# The approximate colorbar limtis for a typical map.
if SOURCE == 'RCW38':
    cbarLWeighted = [[0, 4000], [0, 5000], [0, 500]]
    cbarLWeights = [[0.0, 8.0], [0.0, 14.0], [0.0, 1.2]]
    cbarRWeighted = [[0, 4000], [0, 5000], [0, 500]]
    cbarRWeights = [[0.0, 8.0], [0.0, 14.0], [0.0, 1.2]]
    cbarLUnweighted = [[0, 900], [0, 700], [-100, 700]]
    cbarLUnweightedLog = [[-6.0, 3.0], [-6.0, 3.0], [-5.0, 3.0]]
    cbarLUnweightedZoom = [[0, 900], [0, 700], [-100, 700]]
    cbarLUnweightedZoomLog = [[-6.0, 3.0], [-6.0, 3.0], [-5.0, 3.0]]
    cbarRUnweighted = [[0, 900], [0, 700], [-100, 700]]
    cbarRUnweightedLog = [[-5.0, 3.0], [-5.0, 3.0], [-5.0, 3.0]]
    cbarRUnweightedZoom = [[0, 900], [0, 700], [-100, 700]]
    cbarRUnweightedZoomLog = [[-5.0, 3.0], [-5.0, 3.0], [-5.0, 3.0]]
    cbarSumLR = [[0, 1800], [0, 1400], [-200, 1400]]
    cbarSumLRZoom = [[0, 1800], [0, 1400], [-200, 1400]]
    cbarDiffLR = [[-125, 125], [-100, 100], [-200, 150]]
    cbarDiffLRZoom = [[-125, 125], [-100, 100], [-200, 150]]

    colorbars = [cbarLWeighted, cbarLWeights, cbarRWeighted, cbarRWeights,
                 cbarLUnweighted, cbarLUnweightedLog, cbarLUnweightedZoom, cbarLUnweightedZoomLog,
                 cbarRUnweighted, cbarRUnweightedLog, cbarRUnweightedZoom, cbarRUnweightedZoomLog,
                 cbarSumLR, cbarSumLRZoom, cbarDiffLR, cbarDiffLRZoom]
    
    input_root = '/spt/user/ggeva/RCW38/rcw38_coadd_lr'
    output_root = '/home/ggeva/spt3g/spt3g_software/scratch/ggeva/RCW38/maps/highres/rcw38_coadd_lr/'
    fname_map_sum = '/spt/user/ggeva/map_sums/RCW38/rcw38_coadd_lr/rcw38_coadd_lr_sum.g3'
    mapsumTitle = 'RCW38_Coadd_LR_Split_Combined'

In [ ]:
# Haven't actually determined badobs yet
badobs = ['64927739','65033646'] # Trying to make these plots gave errors for some reason
freqs = ['90GHz', '150GHz', '220GHz']

In [ ]:
# If element is in array, remove the first instance of it.
# (Used to remove the bad observations from the list of observations.)
def RemoveOne(array, element):
    if element in array:
        if array[0] == element:
            return array[1:]
        else:
            return [array[0]] + RemoveOne(array[1:], element)
    else:
        return array

In [ ]:
# Sets up a single subplot appropriately
def SetupSubplot(fig, array, ax, freq, pos, title, noise=False, x0=0, x1=X_MAX, y0=0, y1=Y_MAX, xlabel='Pixel Number', ylabel='Pixel Number'):
    npArray = np.asarray(array)
    if noise:
        vmin=np.percentile(npArray[npArray!=0],10)
        vmax=np.percentile(npArray[npArray!=0],90)
        im = ax.imshow(array, interpolation='none', cmap='jet', vmin=vmin, vmax=vmax)
    # Only commented out until all sources have colorbars set up
    else:
        #vmin=colorbars[pos][freq][0]
        #vmax=colorbars[pos][freq][1]
        im = ax.imshow(array, interpolation='none', cmap='jet')
    ax.set_xlim(x0, x1)
    ax.set_ylim(y0, y1)
    cbar = fig.colorbar(im, ax=ax)
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

In [ ]:
# Creates nine sets of plots from one .g3 file.
# Nine sets come from T, Q, and U data at all three frequency bands.
def CreateSubplot(file, source, title, save=False, outdir='', overwrite=False):      
    prevfiles = []
    for root, dirs, files in os.walk(outdir, topdown=False):
        for f in files:
            prevfiles += [root+f]
            
    for freq in range(len(freqs)):
        data = [frame for frame in core.G3File(file)]
        
        if 'Wpol' in data[freq].keys():
            temppol = True
        else:
            temppol = False
        global POLARIZED
        POLARIZED = POLARIZED
        if temppol != POLARIZED:
            print('Wrong Polarization: ' + output_file)
            POLARIZED = temppol
        
        if POLARIZED:
            leftWeightMap = np.asarray(data[freq]['Wpol'].TT)
            rightWeightMap = np.asarray(data[3+freq]['Wpol'].TT)
        else:
            leftWeightMap = np.asarray(data[freq]['Wunpol'].TT)
            rightWeightMap = np.asarray(data[3+freq]['Wunpol'].TT)
        
        mapmaker.mapmakerutils.RemoveWeightModule(data[freq])
        mapmaker.mapmakerutils.RemoveWeightModule(data[3+freq])
        
        for p in ['T']:
            plotTitle = title + '_' + freqs[freq] + '_' + p
            output_file = outdir + plotTitle + '.png'
            
            if (overwrite == False) and (output_file in prevfiles):
                return
            
            leftMapUnweighted = data[freq][p]
            rightMapUnweighted = data[3+freq][p]
            sumMapUnweighted = leftMapUnweighted + rightMapUnweighted
            diffMapUnweighted = leftMapUnweighted - rightMapUnweighted

            fig, axs = plt.subplots(6, 4, figsize=(30,30))
            fig.subplots_adjust(left=0.02, bottom=0.06, right=0.95, top=0.94, wspace=0.05)

            axLWeights = axs.ravel()[0]
            axRWeights = axs.ravel()[1]
            axLUnweighted = axs.ravel()[4]
            axRUnweighted = axs.ravel()[5]
            axSumLR = axs.ravel()[6]
            axDiffLR = axs.ravel()[7]
            axLUnweightedLog = axs.ravel()[8]
            axRUnweightedLog = axs.ravel()[9]
            axLUnweightedZoom = axs.ravel()[12]
            axRUnweightedZoom = axs.ravel()[13]
            axSumLRZoom = axs.ravel()[14]
            axDiffLRZoom = axs.ravel()[15]
            axLUnweightedZoomLog = axs.ravel()[16]
            axRUnweightedZoomLog = axs.ravel()[17]
            axLNoise = axs.ravel()[20]
            axRNoise = axs.ravel()[21]
            axSumNoise = axs.ravel()[22]
            axDiffNoise = axs.ravel()[23]

            emptyaxs = [2,3,10,11,18,19]


            SetupSubplot(fig=fig, array=leftWeightMap, ax=axLWeights, freq=freq, pos=0,
                         title=source + ' Leftgoing Weight Map ' + freqs[freq])

            SetupSubplot(fig=fig, array=rightWeightMap, ax=axRWeights, freq=freq, pos=1,
                         title=source + ' Rightgoing Weight Map ' + freqs[freq])

            SetupSubplot(fig=fig, array=leftMapUnweighted, ax=axLUnweighted, freq=freq, pos=4,
                         title=source + ' Leftgoing ' + p + ' Map ' + freqs[freq])

            SetupSubplot(fig=fig, array=rightMapUnweighted, ax=axRUnweighted, freq=freq, pos=5,
                         title=source + ' Rightgoing ' + p + ' Map ' + freqs[freq])

            SetupSubplot(fig=fig, array=sumMapUnweighted, ax=axSumLR, freq=freq, pos=6,
                         title=source + ' Left Plus Right ' + p + ' ' + freqs[freq])

            SetupSubplot(fig=fig, array=diffMapUnweighted, ax=axDiffLR, freq=freq, pos=7,
                         title=source + ' Left Minus Right ' + p + ' ' + freqs[freq])

            SetupSubplot(fig=fig, array=np.log10(np.abs(leftMapUnweighted)), ax=axLUnweightedLog, freq=freq, pos=8,
                         title=source + ' Leftgoing Log(' + p + ') Map ' + freqs[freq])

            SetupSubplot(fig=fig, array=np.log10(np.abs(rightMapUnweighted)), ax=axRUnweightedLog, freq=freq, pos=9,
                         title=source + ' Rightgoing Log(' + p + ') Map ' + freqs[freq])

            SetupSubplot(fig=fig, array=leftMapUnweighted, ax=axLUnweightedZoom, freq=freq, pos=12,
                         title='Zoomed-In ' + source + ' Leftgoing ' + p + ' Map ' + freqs[freq],
                         x0=300, x1=400, y0=310, y1=410)

            SetupSubplot(fig=fig, array=rightMapUnweighted, ax=axRUnweightedZoom, freq=freq, pos=13,
                         title='Zoomed-In ' + source + ' Rightgoing ' + p + ' Map ' + freqs[freq],
                         x0=300, x1=400, y0=310, y1=410)

            SetupSubplot(fig=fig, array=sumMapUnweighted, ax=axSumLRZoom, freq=freq, pos=14,
                         title='Zoomed-In '+ source + ' Left Plus Right ' + p + ' ' + freqs[freq],
                         x0=300, x1=400, y0=310, y1=410)

            SetupSubplot(fig=fig, array=diffMapUnweighted, ax=axDiffLRZoom, freq=freq, pos=15,
                         title='Zoomed-In ' + source + ' Left Minus Right ' + p + ' ' + freqs[freq],
                         x0=300, x1=400, y0=310, y1=410)

            SetupSubplot(fig=fig, array=np.log10(np.abs(leftMapUnweighted)), ax=axLUnweightedZoomLog, freq=freq, pos=16,
                         title='Zoomed-In ' + source + ' Leftgoing Log(' + p + ') Map ' + freqs[freq],
                         x0=300, x1=400, y0=310, y1=410)

            SetupSubplot(fig=fig, array=np.log10(np.abs(rightMapUnweighted)), ax=axRUnweightedZoomLog, freq=freq, pos=17,
                         title='Zoomed-In ' + source + ' Rightgoing Log(' + p + ') Map ' + freqs[freq],
                         x0=300, x1=400, y0=310, y1=410)

            SetupSubplot(fig=fig, array=leftMapUnweighted, ax=axLNoise, freq=freq, pos=20, noise=True,
                         title=source + ' Leftgoing ' + p + ' Map (Noise) ' + freqs[freq])

            SetupSubplot(fig=fig, array=rightMapUnweighted, ax=axRNoise, freq=freq, pos=21, noise=True,
                         title=source + ' Rightgoing ' + p + ' Map (Noise) ' + freqs[freq])

            SetupSubplot(fig=fig, array=sumMapUnweighted, ax=axSumNoise, freq=freq, pos=22, noise=True,
                         title=source + ' Left Plus Right ' + p + ' (Noise) ' + freqs[freq])

            SetupSubplot(fig=fig, array=diffMapUnweighted, ax=axDiffNoise, freq=freq, pos=23, noise=True,
                         title=source + ' Left Minus Right ' + p + ' (Noise) ' + freqs[freq])

            for i in emptyaxs:
                fig.delaxes(axs.flatten()[i])

            figTitle = fig.suptitle(plotTitle, fontsize = 'x-large')
            fig.tight_layout()
            figTitle.set_x(0.58)
            figTitle.set_y(1.02)

            if save:
                if outdir != '':
                    plt.savefig(output_file, bbox_inches = 'tight')
                else:
                    print('Plots were not saved. Please specify an output directory.')

In [ ]:
# Make subplots for all observations of a source
def SubplotSource(source, pol=True, save=True, overwrite=False):
    global SOURCE
    global POLARIZED
    SOURCE = source
    POLARIZED = pol
    
    job_root = 'coadd_lr_0.25res'
    if POLARIZED:
        job_root += '_pol'
    input_root = '/spt/user/ggeva/' + SOURCE + '/' + job_root
    output_root = '/home/ggeva/spt3g/spt3g_software/scratch/ggeva/' + SOURCE + '/maps/' + job_root + '/'
    
    observations = []
    fnames = []
    for root, dirs, files in os.walk(input_root, topdown=False):
        for dir in dirs:
            observations += [dir]
            for x, y, z in os.walk(input_root + '/' + dir, topdown=False):
                for file in z:
                    fnames += [x + '/' + file]
                    
    goodobs = observations
    for obs in badobs:
        goodobs = RemoveOne(goodobs, obs)
    
    for obs in goodobs:
        fname = input_root + '/' + obs + '/coadd_lr_0.25res_' + obs + '.g3'
        if fname in fnames:
            if POLARIZED:
                title = SOURCE + '_Coadd_LR_0.25res_pol_' + obs
            else:
                title = SOURCE + '_Coadd_LR_0.25res_' + obs
            
            CreateSubplot(file=fname, source=SOURCE, title=title, save=save, outdir=output_root, overwrite=overwrite)

In [ ]:
# Make subplots for all observations of all sources
Sources = ['CenA-pixelraster', 'PMNJ0210-5101-pixelraster', 'MAT5A-pixelraster', 'RCW38-pixelraster']

for Src in Sources:
    SubplotSource(Src, pol=True, save=True, overwrite=False)

In [ ]:
# Make subplots for coadds (mapsums) of all observations
Sources = ['CenA-pixelraster', 'PMNJ0210-5101-pixelraster', 'MAT5A-pixelraster', 'RCW38-pixelraster']
sources = ['cena-pixelraster', 'pmnj0210-5101-pixelraster', 'mat5a-pixelraster', 'rcw38-pixelraster']
out_root = '/home/ggeva/spt3g/spt3g_software/scratch/ggeva/'
mapsum_root = '/spt/user/ggeva/map_sums/'
job_dir = '/coadd_lr_0.25res_pol/'

for i in range(len(Sources)):
    SOURCE = Sources[i]
    fname = mapsum_root + SOURCE + job_dir + sources[i] + '_coadd_lr_0.25res_sum.g3'
    out_dir = out_root + SOURCE + '/maps' + job_dir
    title = SOURCE + '_Coadd_LR_0.25res_pol_Sum'
    CreateSubplot(fname, SOURCE, title, save=True, outdir=out_dir, overwrite=False)